In [1]:
!pip install xlsxwriter

     |████████████████████████████████| 153kB 7.4MB/s 


In [2]:
import getpass
import requests
import pandas as pd
import xlsxwriter

In [3]:
username=input("NTS username: ")
password = getpass.getpass("NTS password: ")

NTS username: m.van.der.zel@umcg.nl
NTS password: ··········


In [4]:
url = "https://terminologieserver.nl/auth/realms/nictiz/protocol/openid-connect/token"

payload='username='+username+'&password='+password+'&grant_type=password&client_id=cli_client'
headers = { 'Content-Type': 'application/x-www-form-urlencoded' }

response = requests.request("POST", url, headers=headers, data=payload)

token = response.json()
access_token = token["access_token"]

#print(access_token)

In [7]:
url = "https://terminologieserver.nl/fhir/ValueSet/$expand?url=http://snomed.info/sct/15561000146104?fhir_vs=ecl/<362873000"

payload={}
headers = { 'Authorization': 'Bearer ' + access_token }

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

{"resourceType":"ValueSet","language":"en","url":"http://snomed.info/sct/15561000146104/version/20200930?fhir_vs=ecl%2F%3C362873000","name":"SNOMED CT ECL expression","status":"active","experimental":false,"copyright":"This value set includes content from SNOMED CT, which is copyright © 2002+ International Health Terminology Standards Development Organisation (IHTSDO), and distributed by agreement between IHTSDO and HL7. Implementer use of SNOMED CT is not covered by this agreement","expansion":{"extension":[{"url":"http://hl7.org/fhir/StructureDefinition/valueset-unclosed","valueBoolean":true}],"identifier":"572d5543-a8e8-49b7-aab8-56aca7e51dc0","timestamp":"2021-03-24T08:46:44+01:00","total":51,"offset":0,"parameter":[{"name":"version","valueUri":"http://snomed.info/sct|http://snomed.info/sct/15561000146104/version/20200930"},{"name":"count","valueInteger":2147483647},{"name":"offset","valueInteger":0}],"contains":[{"system":"http://snomed.info/sct","code":"362793004","display":"Enti

In [8]:
data = response.json()

print ("Number of concepts in this ValueSet: ", data["expansion"]["total"])
print ()

for entry in data["expansion"]["contains"]:
    print(entry)

Number of concepts in this ValueSet:  51

{'system': 'http://snomed.info/sct', 'code': '362793004', 'display': 'Entire lower leg, from knee to ankle'}
{'system': 'http://snomed.info/sct', 'code': '302553009', 'display': 'geheel abdomen'}
{'system': 'http://snomed.info/sct', 'code': '302538001', 'display': 'gehele bovenarm'}
{'system': 'http://snomed.info/sct', 'code': '302544002', 'display': 'gehele bovenbeen'}
{'system': 'http://snomed.info/sct', 'code': '182245002', 'display': 'gehele bovenste extremiteit'}
{'system': 'http://snomed.info/sct', 'code': '302540006', 'display': 'gehele digitus I van hand'}
{'system': 'http://snomed.info/sct', 'code': '302546000', 'display': 'gehele digitus I van voet'}
{'system': 'http://snomed.info/sct', 'code': '362758006', 'display': 'gehele digitus II van hand'}
{'system': 'http://snomed.info/sct', 'code': '362817008', 'display': 'gehele digitus II van voet'}
{'system': 'http://snomed.info/sct', 'code': '362763005', 'display': 'gehele digitus III va

In [9]:
df = pd.DataFrame(data["expansion"]["contains"])
df = df[['code', 'display']]
print ("Concepts in Refset:", df.size)
df.head(20)

Concepts in Refset: 102


,code,display
0,362793004,"Entire lower leg, from knee to ankle"
1,302553009,geheel abdomen
2,302538001,gehele bovenarm
3,302544002,gehele bovenbeen
4,182245002,gehele bovenste extremiteit
5,302540006,gehele digitus I van hand
6,302546000,gehele digitus I van voet
7,362758006,gehele digitus II van hand
8,362817008,gehele digitus II van voet
9,362763005,gehele digitus III van hand


In [10]:
writer = pd.ExcelWriter('valueset.xlsx')
df.to_excel(writer)
writer.save()